<a href="https://colab.research.google.com/github/bhadreshpsavani/LearningJax/blob/main/Notebooks/JAXvsNumpyModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jax Vs Numpy
In this notebook we will compare simple LinearRegression Model Training using Jax vs Numpy based Model and See how it differs

## Step1. Model Traing with Simple Numpy

In [57]:
import numpy as np
xs = np.random.normal(size=(100,))
noise = np.random.normal(scale=0.1, size=(100,))
ys = xs * 3 - 1 + noise

def model(theta, x):
  W, b = theta
  return W*x + b
  
def update(theta, xs, ys, lr=0.1):
  # Performing Gradient Descent
  W, b = theta
  n = float(len(xs)) # Number of elements in X
  # n = float(len(xs))
  for x, y in zip(xs, ys):
    pred = model(theta, x)
    D_W = (-2/n) *  (x * (y - pred))  # Derivative wrt W
    D_b = (-2/n) *(y - pred)  # Derivative wrt b
    W = W - lr * D_W # Update W
    b = b - lr * D_b  # Update b
  return (W, b)

# initialize single weight and bias variable with one
theta = np.array([1.,1.])

# train model and update weights
for _ in range(1000):
  theta = update(theta, xs, ys)
theta

10 loops, best of 5: 168 ms per loop


Reference:

* https://towardsdatascience.com/linear-regression-using-gradient-descent-97a6c8700931

## Step2. Model Training With Jax

In [58]:
import jax.numpy as jnp
from jax import grad

import numpy as np

xs = np.random.normal(size=(100,))
noise = np.random.normal(scale=0.1, size=(100,))
ys = xs * 3 - 1 + noise

def model(theta, x):
  w, b = theta
  return w*x + b

def loss_fn(theta, x, y):
  # make prediction
  prediction = model(theta, x)
  return jnp.mean((prediction-y)**2) # calculate loss

def update(theta, x, y, lr=0.1):
  return theta - lr * grad(loss_fn)(theta, x, y)

# initialize single weight and bias variable with one
theta = jnp.array([1.,1.])

# train model and update weights
for _ in range(1000):
  theta = update(theta, xs, ys)
theta

1 loop, best of 5: 6.09 s per loop


## Step3. With Pytorch

In [59]:
import torch
from torch.autograd import Variable

import numpy as np
xs = np.random.normal(size=(100,))
noise = np.random.normal(scale=0.1, size=(100,))
ys = xs * 3 - 1 + noise


class LinearRegressionModel(torch.nn.Module):

	def __init__(self):
		super(LinearRegressionModel, self).__init__()
		self.linear = torch.nn.Linear(1, 1) # One in and one out

	def forward(self, x):
		y_pred = self.linear(x)
		return y_pred

# our model
our_model = LinearRegressionModel()

criterion = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.SGD(our_model.parameters(), lr = 0.1)

for epoch in range(1, 1001):
  for x, y in zip(xs, ys):
		# Forward pass: Compute predicted y by passing
		# x to the model
    x_data = Variable(torch.Tensor([[x.item()]]))
    y_data = Variable(torch.Tensor([[y.item()]]))
    pred_y = our_model(x_data)
    # Compute and print loss
    loss = criterion(pred_y, y_data)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  # if epoch%100==0:
  #   print('epoch {}, loss {}'.format(epoch, loss.item()))

for param in our_model.parameters():
  print(param)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Parameter containing:
tensor([[2.9793]], requires_grad=True)
Parameter containing:
tensor([-1.0052], requires_grad=True)
Parameter containing:
tensor([[2.9508]], requires_grad=True)
Parameter containing:
tensor([-0.9880], requires_grad=True)
Parameter containing:
tensor([[3.0377]], requires_grad=True)
Parameter containing:
tensor([-0.9840], requires_grad=True)
Parameter containing:
tensor([[3.0175]], requires_grad=True)
Parameter containing:
tensor([-0.9371], requires_grad=True)
Parameter containing:
tensor([[2.9787]], requires_grad=True)
Parameter containing:
tensor([-1.0217], requires_grad=True)
Parameter containing:
tensor([[3.0295]], requires_grad=True)
Parameter containing:
tensor([-1.0188], requires_grad=True)
1 loop, best of 5: 20.9 s per loop


Reference:     
* https://www.geeksforgeeks.org/linear-regression-using-pytorch/

In [55]:
new_var = Variable(torch.Tensor([[4.0]]))
pred_y = our_model(new_var)
pred_y

tensor([[11.3519]], grad_fn=<AddmmBackward0>)